In [31]:
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver

# Getting the Site ID

In [32]:
response = requests.get('https://api.dineoncampus.com/v1/sites/uchicago/info')
data = response.json()
site_id = data['site']['id']
print("Site ID:", site_id)

Site ID: 60648db131d1ee201bfa84a8


# Info by Slug

In [33]:
slug_request_url = 'https://api.dineoncampus.com/v1/sections/info_by_slug?site_id={}&slug=whats-on-the-menu'.format(site_id)
response = requests.get(slug_request_url)
data = response.json()
print(data)

{'status': 'success', 'request_time': 0.017713481, 'records': 1, 'section': {'id': '60648db431d1ee201bfa84b1', 'name': "What's On The Menu?", 'nodes': None, 'body': 'null', 'show_header': True, 'is_url': False, 'url': None, 'published': True, 'start': '2021-07-21T12:29:45.000-07:00', 'end': '2021-07-21T12:29:45.000-07:00', 'scheduled': False, 'is_file': False, 'file_name': None, 'original_file_name': None, 'global': False, 'change_name': False, 'change_slug': False, 'master': False, 'slug': 'whats-on-the-menu', 'use_banner': False, 'microsite_landing': False, 'banner_text': 'null', 'banner_text_align': 'left', 'banner_text_font': 'inherit', 'tile': False, 'is_published': True, 'no_robots': False, 'script_url': None, 'use_script': False, 'published_status': None, 'master_section': {'id': '5956bc3fee596f675f69f653', 'name': "What's On The Menu?", 'master_slug': 'whats-on-the-menu'}, 'banner': None}}


# All Locations

In [34]:
def building_of_interest(building_name):
    return building_name == 'Bartlett Dining Commons' or building_name == 'Cathey Dining Commons' or building_name == 'Baker Dining Commons' or building_name == 'Woodlawn Dining Commons'

In [35]:
def convert_tuples_to_dict(lst):
    result = {}
    for tuple_item in lst:
        key = tuple_item[1]
        value = tuple_item[0]
        result[key] = value
    return result


In [36]:
all_location_url = 'https://api.dineoncampus.com/v1/locations/all_locations?platform=0&site_id={}&for_menus=true&with_address=false&with_buildings=true'.format(site_id)
response = requests.get(all_location_url)
data = response.json()
# Extract Building IDs
building_ids = []
for building in data['buildings']:
    for location in building['locations']:

        if building_of_interest(location['name']) and location['active']:
            building_ids.append((location['id'], location['name']))

building_ids = convert_tuples_to_dict(building_ids)
print(building_ids)


{'Baker Dining Commons': '618a6caab63f1e2d4442bdf5', 'Cathey Dining Commons': '618a6efbb63f1e2d444389c1', 'Woodlawn Dining Commons': '618a6df9b63f1e2d692b1f5c', 'Bartlett Dining Commons': '618a6f95b63f1e2d3b454065'}


# Periods

In [37]:
response_list = []
for building_name, building_id in building_ids.items():
    period_request_url = 'https://api.dineoncampus.com/v1/location/{}/periods?platform=0&date=2023-5-18'.format(building_id)
    response = requests.get(period_request_url)
    data = response.json()
    if not data['closed']:
        response_list.append((building_name, data))


In [38]:
print(response_list[0][0], "," ,response_list[0][1]['periods'])
print(response_list[0][0], "," , response_list[1][1]['periods'])
print(response_list[0][0], "," ,response_list[2][1]['periods'])

    

Baker Dining Commons , [{'id': '64664c5f351d5305ae3abb93', 'name': 'Breakfast', 'sort_order': 0}, {'id': '64664c5f351d5305ae3abb98', 'name': 'Lunch', 'sort_order': 1}, {'id': '64664c5f351d5305ae3abba9', 'name': 'Dinner', 'sort_order': 2}, {'id': '64664c60351d5305ae3abbbd', 'name': 'Salad Bar', 'sort_order': 3}, {'id': '64664c60351d5305ae3abbc7', 'name': 'Deli Bar', 'sort_order': 4}, {'id': '64664c60351d5305ae3abbc2', 'name': 'Every Day', 'sort_order': 5}, {'id': '64664c60351d5305ae3abbb9', 'name': 'Late Night', 'sort_order': 6}]
Baker Dining Commons , [{'id': '64664be5351d5305c2219dbe', 'name': 'Breakfast', 'sort_order': 0}, {'id': '64664be4351d5305c2219da4', 'name': 'Lunch', 'sort_order': 1}, {'id': '64664be4351d5305c2219db2', 'name': 'Dinner', 'sort_order': 2}, {'id': '64664be5351d5305c2219dc3', 'name': 'Salad Bar', 'sort_order': 3}, {'id': '64664be5351d5305c2219dcd', 'name': 'Deli Bar', 'sort_order': 4}, {'id': '64664be5351d5305c2219dc8', 'name': 'Every Day', 'sort_order': 5}, {'id'

In [39]:
master_dictionary = {}
for building in response_list:
    name = building[0]
    master_dictionary[name] = {}
    for period in building[1]['periods']:
        if period['name'] == 'Lunch' or period['name'] == 'Dinner':
            master_dictionary[name][period['name']] = period['id']
        master_dictionary[name]['location_id'] = building_ids[name]

In [40]:
print(master_dictionary)

{'Baker Dining Commons': {'location_id': '618a6caab63f1e2d4442bdf5', 'Lunch': '64664c5f351d5305ae3abb98', 'Dinner': '64664c5f351d5305ae3abba9'}, 'Cathey Dining Commons': {'location_id': '618a6efbb63f1e2d444389c1', 'Lunch': '64664be4351d5305c2219da4', 'Dinner': '64664be4351d5305c2219db2'}, 'Woodlawn Dining Commons': {'location_id': '618a6df9b63f1e2d692b1f5c', 'Lunch': '64665033351d5305ae3b8486', 'Dinner': '64665034351d5305ae3b84a0'}}


# Example Final Request/Response

In [47]:
# Example of requesting Baker Dinner
baker_location_id = master_dictionary['Baker Dining Commons']['location_id']
print("Location ID", baker_location_id)
baker_dinner_period_id = master_dictionary['Baker Dining Commons']['Dinner']
print("Period ID", baker_dinner_period_id)
baker_dinner_request_url = "https://api.dineoncampus.com/v1/location/{}/periods/{}?platform=0&date=2023-5-18".format(baker_location_id, baker_dinner_period_id)
print(baker_dinner_request_url)


Location ID 618a6caab63f1e2d4442bdf5
Period ID 64664c5f351d5305ae3abba9
https://api.dineoncampus.com/v1/location/618a6caab63f1e2d4442bdf5/periods/64664c5f351d5305ae3abba9?platform=0&date=2023-5-18


In [45]:
response = requests.get(baker_dinner_request_url)
data = response.json()
print(data)

{'status': 'success', 'request_time': 0.048019278, 'records': 1, 'allergen_filter': False, 'menu': {'id': '64664c5c351d5305ae3abb91', 'date': '2023-05-18', 'name': None, 'from_date': None, 'to_date': None, 'periods': {'id': '64664c5f351d5305ae3abba9', 'name': 'Dinner', 'sort_order': 2, 'categories': [{'id': '64664c5f351d5305ae3abbaa', 'name': 'Kitchen', 'sort_order': 0, 'items': [{'id': '6454383b0cf4da340db94471', 'name': 'Roasted NY Strip Steak', 'mrn': '6797', 'rev': '0', 'mrn_full': '6797.0', 'desc': 'Roasted to perfection', 'webtrition_id': 659408625, 'sort_order': 1, 'portion': '3 ounce', 'qty': None, 'ingredients': 'Beef, Burgundy Cooking Wine, Canola Oil, Salt, Garlic, Black Pepper, Rosemary', 'nutrients': [{'id': '6466512ce45d4306a15e6454', 'name': 'Calories', 'value': '270', 'uom': 'kcal', 'value_numeric': '270'}, {'id': '6466512ce45d4306a15e6455', 'name': 'Protein (g)', 'value': '26', 'uom': 'g', 'value_numeric': '26'}, {'id': '6466512ce45d4306a15e6456', 'name': 'Total Carboh